In [1]:
import zipfile, os, re
import pandas as pd
import numpy as np 

In [2]:
_file = os.path.join('ncaa_football_scrapy','Data', 'Data.zip')
zf = zipfile.ZipFile(_file)
filenames = zf.namelist()

In [3]:
filenames

['all-purposeyards.csv',
 'coaches.csv',
 'defense.csv',
 'fumbles.csv',
 'gamebygame_allpurposeyards.csv',
 'gamebygame_defense.csv',
 'gamebygame_fumbles.csv',
 'gamebygame_kicking.csv',
 'gamebygame_kickoffsandkoreturns.csv',
 'gamebygame_participation.csv',
 'gamebygame_passesdefended.csv',
 'gamebygame_passing.csv',
 'gamebygame_punting.csv',
 'gamebygame_puntreturns.csv',
 'gamebygame_receiving.csv',
 'gamebygame_redzone.csv',
 'gamebygame_sacks.csv',
 'gamebygame_scoring.csv',
 'gamebygame_tackles.csv',
 'gamebygame_totaloffense.csv',
 'gamebygame_turnovermargin.csv',
 'history.csv',
 'individualleaders.csv',
 'kicking.csv',
 'kickoffsandkoreturns.csv',
 'participation.csv',
 'passesdefended.csv',
 'passing.csv',
 'punting.csv',
 'puntreturns.csv',
 'receiving.csv',
 'redzone.csv',
 'results.csv',
 'roster.csv',
 'rushing.csv',
 'sacks.csv',
 'scoring.csv',
 'tackles.csv',
 'teamstats.csv',
 'totaloffense.csv',
 'turnovermargin.csv',
 'yearly_allpurposeyards.csv',
 'yearly_defen

# History

In [4]:
history = pd.read_csv(zf.open('history.csv'))

In [5]:
history['year_new'] = history.apply(lambda x: int(x['Year'][:4]), axis = 1)

In [6]:
history_trimmed = history[['Team', "Wins", "Losses", "year_new", "WL"]]

In [7]:
team = 'Nebraska Cornhuskers, Huskers'
target_year = 2016
target_year_min = target_year - 3

In [8]:
history[(history.Team == team) & (history.year_new <= target_year) & (history.year_new >= target_year_min)
       ][['WL']].mean()

WL    0.6345
dtype: float64

# Coach

In [9]:
coaches = pd.read_csv(zf.open('coaches.csv'))

In [10]:
coaches['year_new'] = coaches.apply(lambda x: int(x['Year'][:4]), axis = 1)

In [11]:
coach = 'Bob Diaco'
target_year = 2016

In [12]:
coach_record = coaches[(coaches.Name == coach) & (coaches.year_new <= target_year)][["Name","Wins","Losses","WL","year_new"]].groupby("Name").agg(['sum', 'count', 'mean'])

In [13]:
coach_record.columns = [x[0] + "_" + x[1]  for x in coach_record.columns.values]

In [14]:
coach_record.drop(['Wins_count', 'Wins_mean', 'Losses_count', 'Losses_mean', 'WL_sum', 'WL_count', 
                             'year_new_sum', 'year_new_mean'], axis=1, inplace = True)

In [15]:
coach_record['total_games'] = coach_record.Wins_sum + coach_record.Losses_sum
coach_record.reset_index(inplace = True)

In [16]:
coach_record.head()

,Name,Wins_sum,Losses_sum,WL_mean,year_new_count,total_games
0,Bob Diaco,11.0,26.0,0.293,3,37.0


In [57]:
coaches

,Year,Org,Division,Wins,Losses,Ties,WL,Notes,Name,year_new
0,2007-08,Air Force,FBS,9.0,4.0,0.0,0.692,NaN,Troy Calhoun,2007
1,2008-09,Air Force,FBS,8.0,5.0,0.0,0.615,NaN,Troy Calhoun,2008
2,2009-10,Air Force,FBS,8.0,5.0,0.0,0.615,NaN,Troy Calhoun,2009
3,2010-11,Air Force,FBS,9.0,4.0,0.0,0.692,NaN,Troy Calhoun,2010
4,2011-12,Air Force,FBS,7.0,6.0,0.0,0.538,NaN,Troy Calhoun,2011
5,2012-13,Air Force,FBS,6.0,7.0,0.0,0.462,NaN,Troy Calhoun,2012
6,2013-14,Air Force,FBS,2.0,10.0,0.0,0.167,NaN,Troy Calhoun,2013
7,2014-15,Air Force,FBS,10.0,3.0,0.0,0.769,NaN,Troy Calhoun,2014
8,2015-16,Air Force,FBS,8.0,6.0,0.0,0.571,NaN,Troy Calhoun,2015
9,2016-17,Air Force,FBS,10.0,3.0,0.0,0.769,NaN,Troy Calhoun,2016


# Roster

In [17]:
roster = pd.read_csv(zf.open('roster.csv'))

In [18]:
roster['year_new'] = roster.apply(lambda x: int(x['Year'][:4]), axis = 1)

In [19]:
def yrRecode(data): 
    yr = {'Fr':0, 'So':1, 'Jr':2, 'Sr':3}
    return yr[data]

In [20]:
roster['Yr_N'] = roster.apply(lambda x: yrRecode(x['Yr']) if pd.notnull(x['Yr']) else None, axis = 1)

In [21]:
roster_by_year = roster[['Team', 'Yr', 'year_new', 'GS']].groupby(['Team', 'year_new', 'Yr']).count().reset_index()

In [22]:
roster_games_started = roster[['Team', 'year_new', 'Yr', 'GS']].groupby(['Team', 'year_new', "Yr"]).mean().reset_index()

In [23]:
roster_games_played = roster[['Team', 'year_new', 'Yr', 'GP']].groupby(['Team', 'year_new', "Yr"]).mean().reset_index()

In [24]:
roster_final = pd.merge(roster_games_played, 
                        roster_games_started, 
                        left_on = ['Team', 'year_new', 'Yr'], 
                        right_on = ['Team', 'year_new', 'Yr'],
                        how = 'left'
                       )

In [25]:
roster_final2 = roster_final.set_index(['Team', 'year_new', 'Yr']).unstack().reset_index()

In [26]:
columns = [x[0] if x[0] in ['Team', 'year_new'] else x[0] + "_" + x[1] for x in roster_final2.columns.values]
roster_final2.columns = columns

# Results

In [27]:
game_by_game_results = pd.read_csv(zf.open('results.csv'))

In [28]:
game_by_game_results['homeaway'] = game_by_game_results.apply(lambda x: 'Away' if '@' in x['Opponent'] else 'Home', axis = 1 )

In [29]:
def extractData(data): 
    regexs = {'WL': "[WL]",
              'team': ["\d+\s\-", "\d+"],
              'opponent':["\-\s\d+", '\d+'],
              'OT':["\(\d+OT\)", "\d+"], 
             }
    
    # Win/Loss
    WinLoss = re.search(regexs['WL'], data).group()
    
    # Overtime 
    Overtime = 0
    re_ot = re.search(regexs['OT'][0], data)
    if re_ot:
        Overtime = re.search(regexs['OT'][1], re_ot.group()).group()
        
    # Team
    team_score = None
    re_team = re.search(regexs['team'][0], data)
    if re_team:
        team_score = re.search(regexs['OT'][1], re_team.group()).group()
        
    # Opponent 
    opponent_score = None
    re_opponent = re.search(regexs['opponent'][0], data)
    if re_opponent:
        opponent_score = re.search(regexs['OT'][1], re_opponent.group()).group()
        
    return pd.Series([WinLoss, Overtime, team_score, opponent_score])    

In [30]:
game_by_game_results[['WinLoss','Overtime', 'team_score', 'opponent_score']]=game_by_game_results.apply(lambda x: extractData(x['Result']), axis = 1)

In [31]:
def opponent(data): 
    regex = '[\w\s]+\.?\s\@\s\w+'
    if '@' in data: 
        search = re.search(regex, data) 
        if search: 
            return data.split("@")[0].strip()
        else: 
            return data.replace("@","").strip()
    else: 
        return data

In [32]:
game_by_game_results['opp2'] = game_by_game_results.apply(lambda x: opponent(x['Opponent']), axis = 1)

In [33]:
game_by_game_results.head()

,Date,Opponent,Result,Team,homeaway,WinLoss,Overtime,team_score,opponent_score,opp2
0,08/29/2013,@ UCF,L 7 - 38,Akron Zips,Away,L,0,7,38,UCF
1,09/07/2013,James Madison,W 35 - 33,Akron Zips,Home,W,0,35,33,James Madison
2,09/14/2013,@ Michigan,L 24 - 28,Akron Zips,Away,L,0,24,28,Michigan
3,09/21/2013,Louisiana,L 30 - 35,Akron Zips,Home,L,0,30,35,Louisiana
4,09/28/2013,@ Bowling Green,L 14 - 31,Akron Zips,Away,L,0,14,31,Bowling Green


# Start merging

In [47]:
targetfiles = [file for file in filenames if 'gamebygame' in file ]

In [48]:
dfs = {}
for f in targetfiles: 
    filename = f.split(".")[0]
    dfs[filename] = pd.read_csv(zf.open(f))
    dfs[filename].set_index(['Team', 'Date', 'OffenseDefense'], inplace = True)

In [54]:
master = pd.concat([v for k, v in dfs.items()], axis = 1)
master = master.loc[:, ~master.columns.duplicated()]

In [55]:
master[master.index.get_level_values('OffenseDefense') == 'Offense']

Opponent  \
Team              Date       OffenseDefense                               
Air Force Falcons 08/30/2014 Offense                       Nicholls St.   
                  08/31/2013 Offense                            Colgate   
                  09/03/2016 Offense                  Abilene Christian   
                  09/05/2015 Offense                         Morgan St.   
                  09/06/2014 Offense                          @ Wyoming   
                  09/07/2013 Offense                           Utah St.   
                  09/10/2016 Offense                        Georgia St.   
                  09/12/2015 Offense                       San Jose St.   
                  09/13/2013 Offense                        @ Boise St.   
                  09/13/2014 Offense                      @ Georgia St.   
                  09/19/2015 Offense                     @ Michigan St.   
                  09/21/2013 Offense                            Wyoming   
                  09/24/2016 Offense                         @ Utah St.   
                  09/27/2014 Offense                          Boise St.   
                  09/28/2013 Offense                           @ Nevada   
                  10/01/2016 Offense                               Navy   
                  10/03/2015 Offense                             @ Navy   
                  10/04/2014 Offense                               Navy   
                  10/05/2013 Offense                             @ Navy   
                  10/08/2016 Offense                          @ Wyoming   
                  10/10/2013 Offense                      San Diego St.   
                  10/10/2015 Offense                            Wyoming   
                  10/11/2014 Offense                         @ Utah St.   
                  10/15/2016 Offense         New Mexico @ Dallas, Texas   
                  10/17/2015 Offense                     @ Colorado St.   
                  10/18/2014 Offense                         New Mexico   
                  10/22/2016 Offense                             Hawaii   
                  10/24/2015 Offense                         Fresno St.   
                  10/26/2013 Offense                         Notre Dame   
                  10/28/2016 Offense                       @ Fresno St.   
...                                                                 ...   
Wyoming Cowboys   10/08/2016 Offense                          Air Force   
                  10/10/2015 Offense                        @ Air Force   
                  10/11/2014 Offense                           @ Hawaii   
                  10/12/2013 Offense                         New Mexico   
                  10/17/2015 Offense                             Nevada   
                  10/18/2014 Offense                       San Jose St.   
                  10/19/2013 Offense                       Colorado St.   
                  10/22/2016 Offense                           @ Nevada   
                  10/24/2015 Offense                        @ Boise St.   
                  10/25/2014 Offense                     @ Colorado St.   
                  10/26/2013 Offense                     @ San Jose St.   
                  10/29/2016 Offense                          Boise St.   
                  10/30/2015 Offense                         @ Utah St.   
                  11/01/2014 Offense                       @ Fresno St.   
                  11/05/2016 Offense                           Utah St.   
                  11/07/2014 Offense                           Utah St.   
                  11/07/2015 Offense                       Colorado St.   
                  11/09/2013 Offense                         Fresno St.   
                  11/12/2016 Offense                             @ UNLV   
                  11/14/2015 Offense                    @ San Diego St.   
                  11/16/2013 Offense                        @ Boise St.   
             

In [59]:
set(master.index.get_level_values('Team'))

{'Air Force Falcons',
 'Akron Zips',
 'Alabama Crimson Tide',
 'Appalachian St. Mountaineers',
 'Arizona St. Sun Devils',
 'Arizona Wildcats',
 'Arkansas Razorbacks',
 'Arkansas St. Red Wolves',
 'Army West Point Black Knights',
 'Auburn Tigers',
 'BYU Cougars',
 'Ball St. Cardinals',
 'Baylor Bears',
 'Boise St. Broncos',
 'Boston College Eagles',
 'Bowling Green Falcons',
 'Buffalo Bulls',
 'California Golden Bears',
 'Central Mich. Chippewas',
 'Charlotte 49ers',
 'Cincinnati Bearcats',
 'Clemson Tigers',
 'Coastal Caro. Chanticleers',
 'Colorado Buffaloes',
 'Colorado St. Rams',
 'Duke Blue Devils',
 'East Carolina Pirates',
 'Eastern Mich. Eagles',
 'FIU Panthers',
 'Fla. Atlantic Owls',
 'Florida Gators',
 'Florida St. Seminoles',
 'Fresno St. Bulldogs',
 'Ga. Southern Eagles',
 'Georgia Bulldogs',
 'Georgia St. Panthers',
 'Georgia Tech Yellow Jackets',
 'Hawaii Rainbow Warriors',
 'Houston Cougars',
 'Idaho Vandals',
 'Illinois Fighting Illini',
 'Indiana Hoosiers',
 'Iowa Hawk